In [82]:
import re
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import src.support as sp 

In [83]:
df_cinesa = pd.read_csv("data/Cinesa_cartelera_2023-02-10.csv")
df_yelmo = pd.read_csv("data/Yelmo_cartelera_2023-02-08.csv")
df_golem = pd.read_csv("data/Golem_cartelera_2023-02-14.csv")
df_renoir = pd.read_csv("data/Renoir_cartelera_2023-02-13.csv")
urls_cinesa = pd.read_csv("data/Cinesa_urls.csv")
urls_golem = pd.read_csv("data/Golem_urls.csv")
urls_renoir = pd.read_csv("data/Renoir_urls.csv")

In [84]:
# Split the strings in column '0' of DataFrame into a list of strings, where each string represents a group of information about a cinema
z1 = df_golem["0"].apply(lambda x: x.split("\n"))

# Define a new list of cinema names
new_list = ['Golem Baiona', 'Golem Yamaguchi', 'Golem La Morea', 'Golem Alhóndiga', 'Van Golem', 'Golem Madrid']

# Loop through the new list of cinema names and insert them into the corresponding position in the 'z1' list
for i in range(len(new_list)):
    index = (i)*3  # Calculate the index where the new element should be inserted based on the current position in the loop
    z1.loc[index].append(new_list[i])  # Append the new element at the calculated index in the 'z1' list

In [85]:
# Create a new DataFrame with columns 'cadena', 'cine', 'direccion', 'precio', and 'salas'
df1 = pd.DataFrame(columns=["cadena", "cine", "direccion", "precio", "salas"])

# Define a list of integers 'salas'
salas = [6, 5, 12, 8, 6, 5]

# Define a list of indices that represent the starting position of each group of elements in the original list 'z1'
indices = [i for i in range(0, len(z1), 3)]

# Loop through each group of elements in 'z1' and append a new row to 'df1' with the corresponding values
for i in indices:
    df1 = df1.append({"cadena": "Golem",
                      "cine":z1[i][8:11] ,
                      "direccion": z1[i][:3],
                      "precio": z1[i+2][:13],
                      "salas": salas.pop(0)}, ignore_index=True)

# Loop through each row of 'df1' and modify the values in the 'cine', 'direccion', and 'precio' columns
for i in range(len(df1)):
    df1.at[i, "cine"] = df1.at[i, "cine"][-1]  # Extract the last character of the 'cine' value
    df1.at[i, "direccion"] = " ".join(df1.at[i,"direccion"])  # Join the characters in the 'direccion' value with a space character
    df1.at[i, "precio"] = " ".join(df1.at[i,"precio"])  # Join the characters in the 'precio' value with a space character

In [86]:
# join the two DataFrames column-wise
df1 = df1.join(urls_golem["0"])
df1 = df1.rename(columns={"0": "url"})

In [87]:
# define the regular expression pattern to match the numbers
pattern = r'\d+,\d+'

# define a function that extracts the numbers from a string and returns a dictionary
def extract_prices(input_str):
    # extract all numbers that match the pattern and store them in a list
    numbers = re.findall(pattern, input_str)
    # define a dictionary to map column names to values
    if len(numbers) ==6: # corrected to match the number of keys in the dictionary
        price_dict = {
        "Entrada": numbers[0],
        "Día del espectador": numbers[1],
        "La noche del Domingo": numbers[2],
        "Menores 12 años": numbers[3],
        "Mayores 65 años": numbers[4],
        "Pack Familiar": numbers[5]
        }
    elif len(numbers) ==7:
        price_dict = {
        "Entrada": numbers[0],
        "Día del espectador": numbers[1],
        "La noche del Domingo": numbers[2],
        "Matinales":numbers[3],
        "Menores 12 años": numbers[4],
        "Mayores 65 años": numbers[5],
        "Pack Familiar": numbers[6]
        }
    return price_dict

# apply the function to each row of the "precio" column and store the resulting dictionaries in a new dataframe
new_df = pd.DataFrame(df1["precio"][:4].apply(extract_prices).tolist())

# concatenate the new dataframe with the original dataframe
df1 = pd.concat([df1, new_df], axis=1)

In [88]:
# define the regular expression pattern to match the numbers
pattern = r'\d+,\d+'

# extract all numbers that match the pattern and store them in a list
numbers = re.findall(pattern, df1["precio"][5])

price_dict = {
    "Martes y jueves": numbers[0],  
    "Fin de semana y festivos": numbers[1],
    "Día del espectador": numbers[2],
    "Vuelve al cine": numbers[3],
    "Mayores 65 años": numbers[4],
    "Carné joven": numbers[5],
    "Primera sesión laborables": numbers[6],
    "Última sesión domingos y festivos": numbers[7]
}

# create a new DataFrame with the extracted values
new_df = pd.DataFrame(price_dict, index=[5])

# check if the column name exists in the columns of the original DataFrame
for col in new_df.columns:
    if col in df1.columns:
        df1[col].update(new_df[col])
    else:
        df1 = pd.concat([df1, new_df[col]], axis=1)

In [89]:
# define the regular expression pattern to match the numbers
pattern = r'\d+,\d+'

# extract all numbers that match the pattern and store them in a list
numbers = re.findall(pattern, df1["precio"][4])

price_dict = {
    "Entrada": numbers[0],  
    "Viernes, sábados, domingos y festivos": numbers[1],
    "Día del espectador": numbers[2],
    "Miércoles al cine": numbers[3],
    "La noche del Domingo": numbers[4],
    "Menores 12 años": numbers[5],
    "Mayores 65 años": numbers[6],
    "Pack Familiar": numbers[7]
}

# create a new DataFrame with the extracted values
new_df = pd.DataFrame(price_dict, index=[4])

# check if the column name exists in the columns of the original DataFrame
for col in new_df.columns:
    if col in df1.columns:
        df1[col].update(new_df[col])
    else:
        df1 = pd.concat([df1, new_df[col]], axis=1)

In [90]:
# Select the last 15 columns of the dataframe
last_15_cols = df1.iloc[:, -15:]

# Replace comma with period and convert to float
last_15_cols = last_15_cols.replace(',', '.', regex=True).astype(float)

# Assign the modified columns back to the original dataframe
df1.iloc[:, -15:] = last_15_cols

In [91]:
# Split the strings in column '0' of DataFrame into a list of strings, where each string represents a group of information about a cinema
z = df_renoir["0"].apply(lambda x: x.split("\n"))
# Create a new DataFrame with columns 'cadena', 'cine', 'direccion', 'precio', and 'salas'
df2 = pd.DataFrame(columns=["cadena", "cine", "direccion", "precio", "salas"])

# Define a list of indices that represent the starting position of each group of elements in the original list 'z'
indices = [i for i in range(0, len(z), 3)]

# Loop through each group of elements in 'z' and extract the cinema name, number of rooms, and other information. Then append a new row to 'df2' with the corresponding values
for i in indices:
    cine = z[i][0]  # Extract the cinema name and number of rooms from the first element of the group
    start = cine.find("(")  # Find the start index of the number of rooms
    end = cine.find(")")  # Find the end index of the number of rooms
    salas = cine[start+1:end]  # Extract the number of rooms
    cine = cine[:start-1] + cine[end+1:]  # Extract the cinema name without the number of rooms
    df2 = df2.append({"cadena": "Renoir",
                      "cine": cine,
                      "direccion": z[i][2],
                      "precio": z[2][3:13],
                      "salas": salas},ignore_index=True)  # Append a new row to 'df2' with the corresponding values

# Extracts the numerical part of the "salas" column in the pandas DataFrame df2 and converts it to integer type
df2["salas"] = df2["salas"].str.extract(r"(\d+)", expand=False).astype(int)

# Loops over the rows in the DataFrame df2
for i in range(len(df2)):
    # Extracts the "precio" value from the i-th row and joins the resulting list of strings with a space separator
    df2.at[i, "precio"] = " ".join(df2.at[i, "precio"])

In [92]:
# join the two DataFrames column-wise
df2 = df2.join(urls_renoir["0"])
df2 = df2.rename(columns={"0": "url"})

In [93]:
# Define the regular expression pattern to match the numbers
pattern = r'\d+.\d+'

# Define a function that extracts the numbers from a string and returns a dictionary
def extract_prices(input_str):
    # Extract all numbers that match the pattern and store them in a list
    numbers = re.findall(pattern, input_str)
    # Define a list of positions to filter the extracted numbers
    positions = [0, 2, 4, 6, 8]
    # Filter the extracted numbers to keep only the ones at specified positions
    filtered_values = [numbers[i] for i in positions]
    # Define a dictionary to map column names to values
    price_dict = {
        "Martes y Jueves": numbers[0],
        "Fin de semana y festivos": numbers[1],
        "Día del espectador": numbers[2],
        "Vuelve al cine": numbers[3],
        "Mayores 65 años": numbers[4],
    }
    # Return the resulting dictionary
    return price_dict

# Apply the function to each row of the "precio" column and store the resulting dictionaries in a new dataframe
new_df = pd.DataFrame(df2["precio"].apply(extract_prices).tolist())

# Concatenate the new dataframe with the original dataframe
df2 = pd.concat([df2, new_df], axis=1)

In [94]:
# Select the last 15 columns of the dataframe
last_5_cols = df2.iloc[:, -5:]

# Replace comma with period and convert to float
last_5_cols = last_5_cols.replace(',', '.', regex=True).astype(float)

# Assign the modified columns back to the original dataframe
df2.iloc[:, -5:] = last_5_cols

In [95]:
df_cinesa.drop(["Unnamed: 0"], axis = 1, inplace=True)

In [119]:
# Split the strings in column '0' of DataFrame into a list of strings, where each string represents a group of information about a cinema
z2 = df_cinesa["0"].apply(lambda x: x.split("\n"))

# Create a new DataFrame with columns 'cadena', 'cine', 'direccion', 'precio', and 'salas'
df3 = pd.DataFrame(columns=["cadena", "cine", "direccion", "precio", "salas"])

# Define a list of indices that represent the starting position of each group of elements in the original list 'z2'
indices = list(range(0, len(z2),4))

# Loop through each group of elements in 'z2' and extract the cinema name, number of rooms, and other information.
# Then append a new row to 'df3' with the corresponding values
for i in indices:
    cine = z2[i][0]  # Extract the cinema name and number of rooms from the first element of the group
    direccion = z2[i][-3:] # Extract the last 3 elements from the second element of the group
    precio = z2[i+3]  # Extract the third element of the group
    salas = z2[i+2]  # Extract the second element of the group
    df3 = df3.append({"cadena": "Cinesa",
                      "cine": cine,
                      "direccion": direccion,
                      "precio": precio,
                      "salas": salas},
                     ignore_index=True)  # Append a new row to 'df3' with the corresponding values

# Loop through each row of 'df1' and modify the values in the 'cine', 'direccion', and 'precio' columns
for i in range(len(df3)):
    # df1.at[i, "cine"] = df1.at[i, "cine"][-1]  # Extract the last character of the 'cine' value
    df3.at[i, "direccion"] = " ".join(df3.at[i,"direccion"])  # Join the characters in the 'direccion' value with a space character
    df3.at[i, "precio"] = " ".join(df3.at[i,"precio"])  # Join the characters in the 'precio' value with a space character
    df3.at[i, "salas"] = " ".join(df3.at[i,"salas"])  # Join the characters in the 'precio' value with a space character

# extract the number of salas from each text in the "salas" column
df3["salas"] = df3["salas"].apply(lambda x: re.findall(r'\d+', x)[0])

In [120]:
# define the regular expression pattern to match the numbers
pattern = r'\d+,\d+'

# define a function that extracts the numbers from a string and returns a dictionary
def extract_prices(input_str):
    # extract all numbers that match the pattern and store them in a list
    numbers = re.findall(pattern, input_str)
    if len(numbers) == 8:
        price_dict = {
        "Entrada": numbers[0],  
        "Fin de semana": numbers[1],
        "Matinales": numbers[2],
        "Miércoles al cine": numbers[3],
        "Socio": numbers[4],
        "Carné joven": numbers[5],
        "Mayores 65 años": numbers[6],
        "Menores 12 años": numbers[7]
        }
    elif len(numbers) == 11:
        price_dict = {
        "Entrada": numbers[0],  
        "Fin de semana": numbers[1],
        "Matinales": numbers[2],
        "Miércoles al cine": numbers[3],
        "Socio": numbers[4],
        "Carné joven": numbers[5],
        "Mayores 65 años": numbers[6],
        "Menores 12 años": numbers[7]
        }

    elif len(numbers) == 9:
        price_dict = {
        "Entrada": numbers[0],  
        "Fin de semana": numbers[1],
        "Matinales": numbers[2],
        "Miércoles al cine": numbers[4],
        "Socio": numbers[5],
        "Carné joven": numbers[6],
        "Mayores 65 años": numbers[6],
        "Menores 12 años": numbers[6]
        }    

    elif len(numbers) == 10:
        price_dict = {
        "Entrada": numbers[0],  
        "Fin de semana": numbers[0],
        "Matinales": numbers[1],
        "Miércoles al cine": numbers[3],
        "Socio": numbers[4],
        "Carné joven": numbers[5],
        "Mayores 65 años": numbers[5],
        "Menores 12 años": numbers[6]
        }

    elif len(numbers) == 12:
        price_dict = {
        "Entrada": numbers[0],  
        "Fin de semana": numbers[1],
        "Matinales": numbers[2],
        "Miércoles al cine": numbers[4],
        "Socio": numbers[5],
        "Carné joven": numbers[6],
        "Mayores 65 años": numbers[6],
        "Menores 12 años": numbers[7]
        }   

    elif len(numbers) == 7:
        price_dict = {
        "Entrada": numbers[0],  
        "Fin de semana": numbers[1],
        "Matinales": numbers[2],
        "Miércoles al cine": numbers[3],
        "Socio": numbers[4],
        "Carné joven": numbers[5],
        "Mayores 65 años": numbers[5],
        "Menores 12 años": numbers[5]
        }  

    elif len(numbers) == 13:
        price_dict = {
        "Entrada": numbers[0],  
        "Fin de semana": numbers[1],
        "Matinales": numbers[2],
        "Miércoles al cine": numbers[4],
        "Socio": numbers[5],
        "Carné joven": numbers[6],
        "Mayores 65 años": numbers[6],
        "Menores 12 años": numbers[7]
        } 
    return price_dict

# apply the function to each row of the "precio" column and store the resulting dictionaries in a new dataframe
new_df = pd.DataFrame(df3["precio"].apply(extract_prices).tolist())

# concatenate the new dataframe with the original dataframe
df3 = pd.concat([df3, new_df], axis=1)

In [122]:
# Select the last 15 columns of the dataframe
last_9_cols = df3.iloc[:, -9:]

# Replace comma with period and convert to float
last_9_cols = last_9_cols.replace(',', '.', regex=True).astype(float)

# Assign the modified columns back to the original dataframe
df3.iloc[:, -9:] = last_9_cols

In [123]:
# join the two DataFrames column-wise
df3 = df3.join(urls_cinesa["0"])
df3 = df3.rename(columns={"0": "url"})

In [124]:
# extract the substring containing the postal code and store it in a new column called "CP"
df1["CP"] = df1["direccion"].str.extract(r'(\d{5})')
df2["CP"] = df2["direccion"].str.extract(r'(\d{5})')
df3["CP"] = df3["direccion"].str.extract(r'(\d{5})')

In [125]:
# Store the dataframes into csv
df1.to_csv("data/Golem_Precios.csv")
df2.to_csv("data/Renoir_Precios.csv")
df2.to_csv("data/Cinesa_Precios.csv")

# Merge the data frames with a natural join
merged_df = pd.merge(df1, pd.merge(df2, df3, how='outer'), how='outer')
merged_df.to_csv("data/Merged_Precios.csv")